# Student Loan

# Setup

Run this cell **once** if you are in Google Colab:

In [ ]:
import sys

if "google.colab" in sys.modules:
    !rm -rf sample_data
    !git clone https://github.com/HMellor/personal-finance-uk.git
    !mv personal-finance-uk/* .
    !rm -r personal-finance-uk

Then you can install the Python requirements:

In [ ]:
%pip install -q -r requirements.txt

## Interactive Figure

In [ ]:
from utils.loan import simulate_repayment
from utils.ui import create_inputs, create_figure, plot, observe_children
from dotenv import load_dotenv
import plotly.graph_objects as go
import ipywidgets as widgets
import inspect

load_dotenv()


class InteractiveFigure:
    def __init__(self, process):
        self.process = process
        self.process_kwargs = inspect.signature(self.process).parameters.keys()
        self.plot_kwargs = inspect.signature(plot).parameters.keys()

        # Create inputs and figure
        self.inputs = create_inputs()
        self.figure = create_figure()
        # Initialise the figure
        self.on_change(None, self.figure)

        # Create the controls and display widgets
        self.controls = widgets.VBox(tuple(self.inputs.values()))
        self.display = go.FigureWidget(self.figure)
        # Add callback when change in value is observed for any of the controls
        observe_children(widget=self.controls, callback=self.on_change)

        # Package the controls and the display into the app
        self.app = widgets.VBox((self.controls, self.display))

    # Callback that re-runs the data processing medhot and updates plots
    def on_change(self, _, figure=None):
        # Extract kwargs from inputs
        kwargs = {k: v.value for k, v in self.inputs.items()}
        process_kwargs = {k: v for k, v in kwargs.items() if k in self.process_kwargs}
        plot_kwargs = {k: v for k, v in kwargs.items() if k in self.plot_kwargs}
        # Re-process and plot the new data
        data = self.process(**process_kwargs)
        plot(figure or self.display, data, **plot_kwargs)


InteractiveFigure(simulate_repayment).app